<a href="https://colab.research.google.com/github/kumpaten/masters-thesis-code/blob/main/Glassdoor_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Load the CSV file from your Google Drive
df = pd.read_csv('/content/drive/My Drive/glassdoor_reviews.csv')

# Filter to target firms
target_firms = [
    "Apple", "Microsoft", "Google", "Cisco-Systems", "Oracle",
    "IBM", "SAP", "Facebook", "Accenture", "Salesforce"
]
df_target = df[df['firm'].isin(target_firms)].copy()

# Ensure that the date column is in datetime format and extract the year
df_target['date_review'] = pd.to_datetime(df_target['date_review'], errors='coerce')
df_target['Year'] = df_target['date_review'].dt.year

# Identify the numeric columns for which we want to calculate the average
numeric_cols = [
    'overall_rating',
    'work_life_balance',
    'culture_values',
    'diversity_inclusion',
    'career_opp',
    'comp_benefits',
    'senior_mgmt'
]

# Group by firm and Year, calculating the mean for each numeric column
aggregated_means = (
    df_target.groupby(['firm', 'Year'])[numeric_cols]
             .mean()
             .reset_index()
)

# Calculate the number of rows (i.e., reviews) in each group
row_counts = (
    df_target.groupby(['firm', 'Year'])
             .size()
             .reset_index(name='row_count')
)

# Merge the aggregated means with the row counts
result = pd.merge(aggregated_means, row_counts, on=['firm', 'Year'])

# Display the result
print(result)


In [ ]:
import pandas as pd
from google.colab import files

# Group by 'firm' and 'Year' to calculate the average overall rating and review count
summary_df = df_target.groupby(['firm', 'Year']).agg(
    avg_overall_rating=('overall_rating', 'mean'),
    review_count=('overall_rating', 'count')
).reset_index()

# Display the first few rows of the summary
print(summary_df.head())

In [ ]:
import pandas as pd
import numpy as np

# 1. Add rows for Facebook from 2008 to 2011 if missing
company_to_fix = "Facebook"
missing_years = range(2008, 2012)  # 2008..2011 inclusive

for year in missing_years:
    mask = (summary_df["firm"] == company_to_fix) & (summary_df["Year"] == year)
    if mask.sum() == 0:  # no row found
        new_row_df = pd.DataFrame([{
            "firm": company_to_fix,
            "Year": year,
            "avg_overall_rating": 0.0,
            "review_count": 0
        }])
        summary_df = pd.concat([summary_df, new_row_df], ignore_index=True)

# 2. Sort by company and year
summary_df.sort_values(["firm", "Year"], inplace=True, ignore_index=True)

# 3. Compute a company-specific average rating for each company (m_c)
summary_df["firm_avg"] = summary_df.groupby("firm")["avg_overall_rating"].transform("mean")

# 4. Set prior count C as median of review counts per company (excluding zeros)
def median_nonzero(x):
    nonzero = x[x > 0]
    return int(np.median(nonzero)) if len(nonzero) > 0 else 1

summary_df["C_firm"] = summary_df.groupby("firm")["review_count"].transform(median_nonzero)

# 5. Apply the Bayesian smoothing formula with company-specific average and C
summary_df["smoothed_rating"] = (
    (summary_df["review_count"] * summary_df["avg_overall_rating"] + summary_df["C_firm"] * summary_df["firm_avg"]) /
    (summary_df["review_count"] + summary_df["C_firm"])
)

print("Company-specific median review counts (C_firm) have been applied for smoothing.")

# Display the first few rows of the updated summary
print(summary_df.head())

# 6. Save the summary to a CSV file
output_path = '/content/drive/My Drive/Glassdoor_yearly_review_summary_updated.csv'
summary_df.to_csv(output_path, index=False)

print("The DataFrame now includes missing Facebook rows, is sorted, and uses company-specific smoothed ratings with endogenous median-based C.")